# Road analysis of capital provinces of Veneto

Authors: Biffis Nicola, Pavan Stefano, Trevisi Davide

In [ ]:
!pip install osmnx

In [ ]:
import matplotlib
import pickle
import folium

import matplotlib.pyplot as plt
import seaborn as sns
import osmnx as ox
import networkx as nx

Firstly extract the graph of the intended city from the OSM database with extended boundaries of 5km and an intersection tolerance of 20 meters, based on the experimentation, so the bigger intersections are treated as a single node, and roundabouts are still separated to multiple nodes.

In [ ]:
G_Belluno = ox.graph_from_place('Belluno, 32100, Veneto, Italy', network_type="drive", simplify=True, clean_periphery=True)
G_Padova = ox.graph_from_place('Padova, Veneto, Italy', network_type="drive", simplify=True, clean_periphery=True)
G_Rovigo = ox.graph_from_place('Rovigo, 45100, Veneto, Italy', network_type="drive", simplify=True, clean_periphery=True)
G_Treviso = ox.graph_from_place('Treviso, 31100, Veneto, Italy', network_type="drive", simplify=True, clean_periphery=True)
G_Verona = ox.graph_from_place('Verona, Veneto, Italy', network_type="drive", simplify=True, clean_periphery=True)
G_Vicenza = ox.graph_from_place('Vicenza, 36100, Veneto, Italy', network_type="drive", simplify=True, clean_periphery=True)

Visualize the generated graph on the map

In [ ]:
G_Belluno_proj = ox.project_graph(G_Belluno)
G_Padova_proj = ox.project_graph(G_Padova)
G_Rovigo_proj = ox.project_graph(G_Rovigo)
G_Treviso_proj = ox.project_graph(G_Treviso)
G_Verona_proj = ox.project_graph(G_Verona)
G_Vicenza_proj = ox.project_graph(G_Vicenza)

In [ ]:
G_Belluno_final = ox.consolidate_intersections(G_Belluno_proj , rebuild_graph=True, tolerance=15, dead_ends=False)
G_Padova_final = ox.consolidate_intersections(G_Padova_proj , rebuild_graph=True, tolerance=15, dead_ends=False)
G_Rovigo_final = ox.consolidate_intersections(G_Rovigo_proj , rebuild_graph=True, tolerance=15, dead_ends=False)
G_Treviso_final = ox.consolidate_intersections(G_Treviso_proj , rebuild_graph=True, tolerance=15, dead_ends=False)
G_Verona_final = ox.consolidate_intersections(G_Verona_proj , rebuild_graph=True, tolerance=15, dead_ends=False)
G_Vicenza_final = ox.consolidate_intersections(G_Vicenza_proj , rebuild_graph=True, tolerance=15, dead_ends=False)

##Belluno

In [ ]:
nodes, streets = ox.graph_to_gdfs(G_Belluno_final)
m = folium.Map(location=ox.geocode("Belluno"))
folium.GeoJson(streets).add_to(m)
folium.GeoJson(nodes, marker=folium.CircleMarker(radius=3)).add_to(m)
m

Calculate the betwenness centrality, save it and then visualize the new graph on the map

In [ ]:
def eval_centrality(g, edge_centrality_fn, node_centrality_fn, weight, inplace=True):
    if not inplace:
        g = g.copy()
    nodes_centrality = node_centrality_fn(g, weight=weight)
    edges_centrality = edge_centrality_fn(g, weight=weight)

    for (u, v, i), centrality in edges_centrality.items():
        g[u][v][i]['centrality'] = centrality

    for node, centrality in nodes_centrality.items():
        g.nodes[node]['centrality'] = centrality

    return g

In [ ]:
G_betweenness_Belluno = eval_centrality(G_Belluno_final, nx.edge_betweenness_centrality, nx.betweenness_centrality, 'travel_time')
#pickle.dump(G_betweenness_Belluno, open('graphs/padua_large_betweenness.pickle', 'wb'))
#G_betweenness_Belluno = pickle.load(open('graphs/padua_large_betweenness.pickle', 'rb'))

Get the most popular roads of the intended city and plot the graphs of cumulative betweenness centrality about the streets and the junctions

In [ ]:
def get_most_popular_roads(G: nx.MultiDiGraph, n):
    return [x[2] for x in sorted(G.edges(data=True), key=lambda x: -1 * x[2]['centrality'])[:n]]

def concat_road_names(x):
    if 'name' in x:
        name = x['name']
        if isinstance(name, list):
            return "-".join(name)
        return name
    elif 'ref' in x:
        return x['ref']

In [ ]:
most_popular_roads = get_most_popular_roads(G_betweenness_Belluno, n=50)
set([concat_road_names(x) for x in most_popular_roads if 'ref' in x or 'name' in x])

In [ ]:
nodes, streets = ox.graph_to_gdfs(G_betweenness_Belluno)

In [ ]:
fig, ax = plt.subplots(nrows=2)

sns.ecdfplot(streets, x='centrality', ax=ax[0])
ax[0].set_xscale('log')
ax[0].set_title('Street centrality cumulative distribution')

sns.ecdfplot(nodes, x='centrality', ax=ax[1])
ax[1].set_xscale('log')
ax[1].set_title('Junctions centrality cumulative distribution')

fig.suptitle('Cumulative betweenness centrality of Belluno', fontsize=16)
fig.set_size_inches(10, 10)
fig.tight_layout()
plt.savefig('figures/cdf-belluno.png', dpi=300)

Select the top 2% of the streets ranked by betwenness centrality score and visualize them on the map

In [ ]:
nodes_th = nodes['centrality'].quantile(q=0.98)
streets_th = streets['centrality'].quantile(q=0.98)
nodes_th, streets_th
small_nodes = nodes[nodes['centrality'] >= nodes_th]
small_streets = streets[streets['centrality'] >= streets_th]
m = folium.Map(location=ox.geocode('Belluno'))
eg=folium.GeoJson(small_streets).add_to(m)
ng=folium.GeoJson(small_nodes, marker=folium.CircleMarker(radius=3)).add_to(m)
folium.features.GeoJsonPopup(fields=['ref', 'name', 'centrality']).add_to(eg)
folium.features.GeoJsonPopup(fields=['centrality']).add_to(ng)
eg.add_to(m)
ng.add_to(m)
m

##Padua

In [ ]:
nodes, streets = ox.graph_to_gdfs(G_Padova_final)
m = folium.Map(location=ox.geocode("Padua"))
folium.GeoJson(streets).add_to(m)
folium.GeoJson(nodes, marker=folium.CircleMarker(radius=3)).add_to(m)
m

Calculate the betwenness centrality, save it and then visualize the new graph on the map

In [ ]:
def eval_centrality(g, edge_centrality_fn, node_centrality_fn, weight, inplace=True):
    if not inplace:
        g = g.copy()
    nodes_centrality = node_centrality_fn(g, weight=weight)
    edges_centrality = edge_centrality_fn(g, weight=weight)

    for (u, v, i), centrality in edges_centrality.items():
        g[u][v][i]['centrality'] = centrality

    for node, centrality in nodes_centrality.items():
        g.nodes[node]['centrality'] = centrality

    return g

In [ ]:
G_betweenness_Padova = eval_centrality(G_Padova_final, nx.edge_betweenness_centrality, nx.betweenness_centrality, 'travel_time')
#pickle.dump(G_betweenness_Padova, open('graphs/padua_large_betweenness.pickle', 'wb'))
#G_betweenness_Padova = pickle.load(open('graphs/padua_large_betweenness.pickle', 'rb'))

Get the most popular roads of the intended city and plot the graphs of cumulative betweenness centrality about the streets and the junctions

In [ ]:
def get_most_popular_roads(G: nx.MultiDiGraph, n):
    return [x[2] for x in sorted(G.edges(data=True), key=lambda x: -1 * x[2]['centrality'])[:n]]

def concat_road_names(x):
    if 'name' in x:
        name = x['name']
        if isinstance(name, list):
            return "-".join(name)
        return name
    elif 'ref' in x:
        return x['ref']

In [ ]:
most_popular_roads = get_most_popular_roads(G_betweenness_Padova, n=50)
set([concat_road_names(x) for x in most_popular_roads if 'ref' in x or 'name' in x])

In [ ]:
nodes, streets = ox.graph_to_gdfs(G_betweenness_Padova)

In [ ]:
fig, ax = plt.subplots(nrows=2)

sns.ecdfplot(streets, x='centrality', ax=ax[0])
ax[0].set_xscale('log')
ax[0].set_title('Street centrality cumulative distribution')

sns.ecdfplot(nodes, x='centrality', ax=ax[1])
ax[1].set_xscale('log')
ax[1].set_title('Junctions centrality cumulative distribution')

fig.suptitle('Cumulative betweenness centrality of Padua', fontsize=16)
fig.set_size_inches(10, 10)
fig.tight_layout()
plt.savefig('figures/cdf-padua.png', dpi=300)

Select the top 2% of the streets ranked by betwenness centrality score and visualize them on the map

In [ ]:
nodes_th = nodes['centrality'].quantile(q=0.98)
streets_th = streets['centrality'].quantile(q=0.98)
nodes_th, streets_th
small_nodes = nodes[nodes['centrality'] >= nodes_th]
small_streets = streets[streets['centrality'] >= streets_th]
m = folium.Map(location=ox.geocode('Padua'))
eg=folium.GeoJson(small_streets).add_to(m)
ng=folium.GeoJson(small_nodes, marker=folium.CircleMarker(radius=3)).add_to(m)
folium.features.GeoJsonPopup(fields=['ref', 'name', 'centrality']).add_to(eg)
folium.features.GeoJsonPopup(fields=['centrality']).add_to(ng)
eg.add_to(m)
ng.add_to(m)
m

##Rovigo

In [ ]:
nodes, streets = ox.graph_to_gdfs(G_Rovigo_final)
m = folium.Map(location=ox.geocode("Rovigo"))
folium.GeoJson(streets).add_to(m)
folium.GeoJson(nodes, marker=folium.CircleMarker(radius=3)).add_to(m)
m

Calculate the betwenness centrality, save it and then visualize the new graph on the map

In [ ]:
def eval_centrality(g, edge_centrality_fn, node_centrality_fn, weight, inplace=True):
    if not inplace:
        g = g.copy()
    nodes_centrality = node_centrality_fn(g, weight=weight)
    edges_centrality = edge_centrality_fn(g, weight=weight)

    for (u, v, i), centrality in edges_centrality.items():
        g[u][v][i]['centrality'] = centrality

    for node, centrality in nodes_centrality.items():
        g.nodes[node]['centrality'] = centrality

    return g

In [ ]:
G_betweenness_Rovigo = eval_centrality(G_Rovigo_final, nx.edge_betweenness_centrality, nx.betweenness_centrality, 'travel_time')
#pickle.dump(G_betweenness_Rovigo, open('graphs/padua_large_betweenness.pickle', 'wb'))
#G_betweenness_Rovigo = pickle.load(open('graphs/padua_large_betweenness.pickle', 'rb'))

Get the most popular roads of the intended city and plot the graphs of cumulative betweenness centrality about the streets and the junctions

In [ ]:
def get_most_popular_roads(G: nx.MultiDiGraph, n):
    return [x[2] for x in sorted(G.edges(data=True), key=lambda x: -1 * x[2]['centrality'])[:n]]

def concat_road_names(x):
    if 'name' in x:
        name = x['name']
        if isinstance(name, list):
            return "-".join(name)
        return name
    elif 'ref' in x:
        return x['ref']

In [ ]:
most_popular_roads = get_most_popular_roads(G_betweenness_Rovigo, n=50)
set([concat_road_names(x) for x in most_popular_roads if 'ref' in x or 'name' in x])

In [ ]:
nodes, streets = ox.graph_to_gdfs(G_betweenness_Rovigo)

In [ ]:
fig, ax = plt.subplots(nrows=2)

sns.ecdfplot(streets, x='centrality', ax=ax[0])
ax[0].set_xscale('log')
ax[0].set_title('Street centrality cumulative distribution')

sns.ecdfplot(nodes, x='centrality', ax=ax[1])
ax[1].set_xscale('log')
ax[1].set_title('Junctions centrality cumulative distribution')

fig.suptitle('Cumulative betweenness centrality of Rovigo', fontsize=16)
fig.set_size_inches(10, 10)
fig.tight_layout()
plt.savefig('figures/cdf-rovigo.png', dpi=300)

Select the top 2% of the streets ranked by betwenness centrality score and visualize them on the map

In [ ]:
nodes_th = nodes['centrality'].quantile(q=0.98)
streets_th = streets['centrality'].quantile(q=0.98)
nodes_th, streets_th
small_nodes = nodes[nodes['centrality'] >= nodes_th]
small_streets = streets[streets['centrality'] >= streets_th]
m = folium.Map(location=ox.geocode('Rovigo'))
eg=folium.GeoJson(small_streets).add_to(m)
ng=folium.GeoJson(small_nodes, marker=folium.CircleMarker(radius=3)).add_to(m)
folium.features.GeoJsonPopup(fields=['ref', 'name', 'centrality']).add_to(eg)
folium.features.GeoJsonPopup(fields=['centrality']).add_to(ng)
eg.add_to(m)
ng.add_to(m)
m

##Treviso

In [ ]:
nodes, streets = ox.graph_to_gdfs(G_Treviso_final)
m = folium.Map(location=ox.geocode("Treviso"))
folium.GeoJson(streets).add_to(m)
folium.GeoJson(nodes, marker=folium.CircleMarker(radius=3)).add_to(m)
m

Calculate the betwenness centrality, save it and then visualize the new graph on the map

In [ ]:
def eval_centrality(g, edge_centrality_fn, node_centrality_fn, weight, inplace=True):
    if not inplace:
        g = g.copy()
    nodes_centrality = node_centrality_fn(g, weight=weight)
    edges_centrality = edge_centrality_fn(g, weight=weight)

    for (u, v, i), centrality in edges_centrality.items():
        g[u][v][i]['centrality'] = centrality

    for node, centrality in nodes_centrality.items():
        g.nodes[node]['centrality'] = centrality

    return g

In [ ]:
G_betweenness_Treviso = eval_centrality(G_Treviso_final, nx.edge_betweenness_centrality, nx.betweenness_centrality, 'travel_time')
#pickle.dump(G_betweenness_Treviso, open('graphs/padua_large_betweenness.pickle', 'wb'))
#G_betweenness_Treviso = pickle.load(open('graphs/padua_large_betweenness.pickle', 'rb'))

Get the most popular roads of the intended city and plot the graphs of cumulative betweenness centrality about the streets and the junctions

In [ ]:
def get_most_popular_roads(G: nx.MultiDiGraph, n):
    return [x[2] for x in sorted(G.edges(data=True), key=lambda x: -1 * x[2]['centrality'])[:n]]

def concat_road_names(x):
    if 'name' in x:
        name = x['name']
        if isinstance(name, list):
            return "-".join(name)
        return name
    elif 'ref' in x:
        return x['ref']

In [ ]:
most_popular_roads = get_most_popular_roads(G_betweenness_Treviso, n=50)
set([concat_road_names(x) for x in most_popular_roads if 'ref' in x or 'name' in x])

In [ ]:
nodes, streets = ox.graph_to_gdfs(G_betweenness_Treviso)

In [ ]:
fig, ax = plt.subplots(nrows=2)

sns.ecdfplot(streets, x='centrality', ax=ax[0])
ax[0].set_xscale('log')
ax[0].set_title('Street centrality cumulative distribution')

sns.ecdfplot(nodes, x='centrality', ax=ax[1])
ax[1].set_xscale('log')
ax[1].set_title('Junctions centrality cumulative distribution')

fig.suptitle('Cumulative betweenness centrality of Treviso', fontsize=16)
fig.set_size_inches(10, 10)
fig.tight_layout()
plt.savefig('figures/cdf-treviso.png', dpi=300)

Select the top 2% of the streets ranked by betwenness centrality score and visualize them on the map

In [ ]:
nodes_th = nodes['centrality'].quantile(q=0.98)
streets_th = streets['centrality'].quantile(q=0.98)
nodes_th, streets_th
small_nodes = nodes[nodes['centrality'] >= nodes_th]
small_streets = streets[streets['centrality'] >= streets_th]
m = folium.Map(location=ox.geocode('Treviso'))
eg=folium.GeoJson(small_streets).add_to(m)
ng=folium.GeoJson(small_nodes, marker=folium.CircleMarker(radius=3)).add_to(m)
folium.features.GeoJsonPopup(fields=['ref', 'name', 'centrality']).add_to(eg)
folium.features.GeoJsonPopup(fields=['centrality']).add_to(ng)
eg.add_to(m)
ng.add_to(m)
m

##Verona

In [ ]:
nodes, streets = ox.graph_to_gdfs(G_Verona_final)
m = folium.Map(location=ox.geocode("Verona"))
folium.GeoJson(streets).add_to(m)
folium.GeoJson(nodes, marker=folium.CircleMarker(radius=3)).add_to(m)
m

Calculate the betwenness centrality, save it and then visualize the new graph on the map

In [ ]:
def eval_centrality(g, edge_centrality_fn, node_centrality_fn, weight, inplace=True):
    if not inplace:
        g = g.copy()
    nodes_centrality = node_centrality_fn(g, weight=weight)
    edges_centrality = edge_centrality_fn(g, weight=weight)

    for (u, v, i), centrality in edges_centrality.items():
        g[u][v][i]['centrality'] = centrality

    for node, centrality in nodes_centrality.items():
        g.nodes[node]['centrality'] = centrality

    return g

In [ ]:
G_betweenness_Verona = eval_centrality(G_Verona_final, nx.edge_betweenness_centrality, nx.betweenness_centrality, 'travel_time')
#pickle.dump(G_betweenness_Verona, open('graphs/padua_large_betweenness.pickle', 'wb'))
#G_betweenness_Verona = pickle.load(open('graphs/padua_large_betweenness.pickle', 'rb'))

Get the most popular roads of the intended city and plot the graphs of cumulative betweenness centrality about the streets and the junctions

In [ ]:
def get_most_popular_roads(G: nx.MultiDiGraph, n):
    return [x[2] for x in sorted(G.edges(data=True), key=lambda x: -1 * x[2]['centrality'])[:n]]

def concat_road_names(x):
    if 'name' in x:
        name = x['name']
        if isinstance(name, list):
            return "-".join(name)
        return name
    elif 'ref' in x:
        return x['ref']

In [ ]:
most_popular_roads = get_most_popular_roads(G_betweenness_Verona, n=50)
set([concat_road_names(x) for x in most_popular_roads if 'ref' in x or 'name' in x])

In [ ]:
nodes, streets = ox.graph_to_gdfs(G_betweenness_Verona)

In [ ]:
fig, ax = plt.subplots(nrows=2)

sns.ecdfplot(streets, x='centrality', ax=ax[0])
ax[0].set_xscale('log')
ax[0].set_title('Street centrality cumulative distribution')

sns.ecdfplot(nodes, x='centrality', ax=ax[1])
ax[1].set_xscale('log')
ax[1].set_title('Junctions centrality cumulative distribution')

fig.suptitle('Cumulative betweenness centrality of Verona', fontsize=16)
fig.set_size_inches(10, 10)
fig.tight_layout()
plt.savefig('figures/cdf-verona.png', dpi=300)

Select the top 2% of the streets ranked by betwenness centrality score and visualize them on the map

In [ ]:
nodes_th = nodes['centrality'].quantile(q=0.98)
streets_th = streets['centrality'].quantile(q=0.98)
nodes_th, streets_th
small_nodes = nodes[nodes['centrality'] >= nodes_th]
small_streets = streets[streets['centrality'] >= streets_th]
m = folium.Map(location=ox.geocode('Verona'))
eg=folium.GeoJson(small_streets).add_to(m)
ng=folium.GeoJson(small_nodes, marker=folium.CircleMarker(radius=3)).add_to(m)
folium.features.GeoJsonPopup(fields=['ref', 'name', 'centrality']).add_to(eg)
folium.features.GeoJsonPopup(fields=['centrality']).add_to(ng)
eg.add_to(m)
ng.add_to(m)
m

##Vicenza

In [ ]:
nodes, streets = ox.graph_to_gdfs(G_Vicenza_final)
m = folium.Map(location=ox.geocode("Vicenza"))
folium.GeoJson(streets).add_to(m)
folium.GeoJson(nodes, marker=folium.CircleMarker(radius=3)).add_to(m)
m

Calculate the betwenness centrality, save it and then visualize the new graph on the map

In [ ]:
def eval_centrality(g, edge_centrality_fn, node_centrality_fn, weight, inplace=True):
    if not inplace:
        g = g.copy()
    nodes_centrality = node_centrality_fn(g, weight=weight)
    edges_centrality = edge_centrality_fn(g, weight=weight)

    for (u, v, i), centrality in edges_centrality.items():
        g[u][v][i]['centrality'] = centrality

    for node, centrality in nodes_centrality.items():
        g.nodes[node]['centrality'] = centrality

    return g

In [ ]:
G_betweenness_Vicenza = eval_centrality(G_Vicenza_final, nx.edge_betweenness_centrality, nx.betweenness_centrality, 'travel_time')
#pickle.dump(G_betweenness_Vicenza, open('graphs/padua_large_betweenness.pickle', 'wb'))
#G_betweenness_Vicenza = pickle.load(open('graphs/padua_large_betweenness.pickle', 'rb'))

Get the most popular roads of the intended city and plot the graphs of cumulative betweenness centrality about the streets and the junctions

In [ ]:
def get_most_popular_roads(G: nx.MultiDiGraph, n):
    return [x[2] for x in sorted(G.edges(data=True), key=lambda x: -1 * x[2]['centrality'])[:n]]

def concat_road_names(x):
    if 'name' in x:
        name = x['name']
        if isinstance(name, list):
            return "-".join(name)
        return name
    elif 'ref' in x:
        return x['ref']

In [ ]:
most_popular_roads = get_most_popular_roads(G_betweenness_Vicenza, n=50)
set([concat_road_names(x) for x in most_popular_roads if 'ref' in x or 'name' in x])

In [ ]:
nodes, streets = ox.graph_to_gdfs(G_betweenness_Vicenza)

In [ ]:
fig, ax = plt.subplots(nrows=2)

sns.ecdfplot(streets, x='centrality', ax=ax[0])
ax[0].set_xscale('log')
ax[0].set_title('Street centrality cumulative distribution')

sns.ecdfplot(nodes, x='centrality', ax=ax[1])
ax[1].set_xscale('log')
ax[1].set_title('Junctions centrality cumulative distribution')

fig.suptitle('Cumulative betweenness centrality of Vicenza', fontsize=16)
fig.set_size_inches(10, 10)
fig.tight_layout()
plt.savefig('figures/cdf-vicenza.png', dpi=300)

Select the top 2% of the streets ranked by betwenness centrality score and visualize them on the map

In [ ]:
nodes_th = nodes['centrality'].quantile(q=0.98)
streets_th = streets['centrality'].quantile(q=0.98)
nodes_th, streets_th
small_nodes = nodes[nodes['centrality'] >= nodes_th]
small_streets = streets[streets['centrality'] >= streets_th]
m = folium.Map(location=ox.geocode('Vicenza'))
eg=folium.GeoJson(small_streets).add_to(m)
ng=folium.GeoJson(small_nodes, marker=folium.CircleMarker(radius=3)).add_to(m)
folium.features.GeoJsonPopup(fields=['ref', 'name', 'centrality']).add_to(eg)
folium.features.GeoJsonPopup(fields=['centrality']).add_to(ng)
eg.add_to(m)
ng.add_to(m)
m